In [1]:
import pandas as pd

In [2]:
data = pd.read_parquet('../data/curated/twitter_with_sent.parquet')

In [3]:
data['neu_score'] = 1-data['pos_score']-data['neg_score']

In [4]:
data = data.drop('sentiment', axis = 1)

In [5]:
data['p>n'] = data['pos_score']>data['neg_score']
data['p>nu'] = data['pos_score']>data['neu_score']
data['nu>n'] = data['neu_score']>data['neg_score']

# p: p>n and p > nu
# nu: nu>n and !p>nu
# neg: !p>n and !nu>n

p: p>n and p > nu
nu: nu>n and !p>nu
neg: !p>n and !nu>n

In [6]:
import geopandas as gpd
from shapely.geometry import Point

# Load the GeoPandas polygon data
polygon_gdf = gpd.read_file("../data/raw/Geography/2021_ELB_region.shp")


/Applications/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [7]:
def get_Electorate(coord):
    for idx, polygon in polygon_gdf.iterrows():
        if polygon.geometry.contains(coord):
            return polygon['Elect_div']
    
    return None

In [5]:
import subprocess

In [6]:
subprocess.run(f'curl -X PUT http://172.26.131.22:5984/test_bulk -u admin:password', shell=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

{"ok":true}


100    12  100    12    0     0     37      0 --:--:-- --:--:-- --:--:--    37


CompletedProcess(args='curl -X PUT http://172.26.131.22:5984/test_bulk -u admin:password', returncode=0)

In [12]:
import json

In [17]:
list_of_doc_dict = []

for id, row in data.iterrows():

    # coord
    coord = Point(row["x_cent"], row["y_cent"])
    electorate = get_Electorate(coord)
    print(electorate)

    if row['p>n']:
        if row['p>nu']:
            sentiment = 1
        else:
            sentiment = 0
    elif row['p>nu']:
        if row['p>n']:
            sentiment = 1
        else:
            sentiment = -1
    elif row['nu>n']:
        if row['p>nu']:
            sentiment = 1
        else:
            sentiment = 0


    text = row['text']
    text = text.replace("'", "\\'")

    
    row_json = {"id": f"{row['_id']}", "author_id": f"{row['author_id']}", "date": f"{row['date']}", "text": text, 
            "is_political": row['is_political'], "neg_score": row['neg_score'], "neu_score": row['neu_score'], 
            "pos_score": row['pos_score'], "compound_score": row['compound_score'], "sentiment": sentiment, 
            "electorate": f"{electorate}"}
    

    list_of_doc_dict.append(row_json)

    if (id+1)%100 == 0:
        

        out = {"docs": list_of_doc_dict}


        json_data = json.dumps(out).encode('utf-8')

        subprocess.run(['curl', '-X', 'POST', f'http://172.26.131.22:5984/test_new/_bulk_docs', '--header', 'Content-Type: application/json', '--data-binary', '@-', '-u', 'admin:password'], input=json_data)
        
        list_of_doc_dict = []

        

Capricornia
Fadden
Groom
Burt
McMahon
Goldstein
Lilley
Goldstein
Lilley
McMahon
McMahon
McMahon
McMahon
McMahon
McMahon
Lilley
Lilley
Lingiari
Gilmore
McMahon
Grey
Grey
Boothby
Goldstein
McMahon
None
Lingiari
Boothby
Burt
Franklin
Goldstein
Goldstein
Boothby
Goldstein
Burt
Calare
McMahon
Braddon
Braddon
Fadden
Goldstein
O'Connor
O'Connor
Boothby
Goldstein
Lilley
McMahon
Goldstein
Lilley
McMahon
McMahon
McMahon
Goldstein
Goldstein
Cooper
Goldstein
Goldstein
Lalor
Goldstein
Goldstein
Goldstein
Goldstein
Goldstein
Goldstein
Goldstein
Lalor
Goldstein
McMahon
McMahon
Cunningham
Boothby
McMahon
McMahon
McMahon
Groom
Burt
Grey
La Trobe
McMahon
McMahon
McMahon
Calare
Goldstein
Goldstein
Lilley
Boothby
Lilley
Lilley
McMahon
Goldstein
Goldstein
Nicholls
Lilley
Shortland
McMahon
Goldstein
Burt
Grey
Goldstein
Lilley
[{"ok":true,"id":"ab53a8d504bc5f849e65e139b44e29e9","rev":"1-1db53322577b1ff025babb32ca4c6130"},{"ok":true,"id":"ab53a8d504bc5f849e65e139b44e335b","rev":"1-e3bcb98e1e6a91df5256c479c3c3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 53972  100  9502  100 44470  91733   419k --:--:-- --:--:-- --:--:--  511k


In [18]:
out

{'docs': [{'id': '1491567527322808321',
   'author_id': '45472006',
   'date': '2022-02-10',
   'text': 'My life is hitting a big change soon. Keen to make a new Champagne vid for everyone! Just have to shake my life up. In the process! Excited! https://t.co/EeJH7bfr71',
   'is_political': 0,
   'neg_score': 0.052,
   'neu_score': 0.704,
   'pos_score': 0.244,
   'compound_score': 0.7412,
   'sentiment': 0,
   'electorate': 'Capricornia'},
  {'id': '1491693811663515654',
   'author_id': '952342256823943168',
   'date': '2022-02-10',
   'text': '@TobyRayEnglish @MikeDel21893959 @aSinister @biffmack @brandondaly2018 @fishyfish67 @ChrisBBacon3 @6lennybravo @insane_voice @JsharkJill @TheDisproof @GaryWHall @MarshallCd @FriendsOScience @cieloyla @LouisCy72344053 @JohnDublin10 @The_Doctor_Why @RustyAway @glinch72 @MedBennett @Narvuntien @robhon_ @TroyCross24 @Barbarajdurkin @CanadianSence @jpgcrowley @Anvndarnamn5 @Michael_56 @GeraldKutney @ponycar06 @Jonnyporkchop @PoeBrianL @DawnTJ90 @Mich